In [1]:
# This is super simple importing the libararies needed.
import numpy as np
import random
import csv
import math
import random
import math
import heapq
import numpy as np

In [2]:
'''
This is what we use to convert a variable to a binary sring.
Takes in the key, then uses format to format it to a 16 len binary string.
Important to not that it is explicitly a string, not an int.
'''
def hash_funtion(key):
    
    return  '{0:016b}'.format(key)

'''
Here we define a bucket
We set the ID for the bucket, this is defined when making the bucket
local depth defines the ammount of values for each bucket
index is the values stored within
empty spaces defines how much room is left in the bucket
'''
class Bucket:
    
    def __init__(self,local_depth,index,empty_spaces,id):
        
        self.id = id
        self.local_depth = local_depth
        self.index = index
        self.empty_spaces = empty_spaces

'''
This is the central data strcuture that holds the directories that then hold the buckets
global_depth how many bits since 2^Global depth
    e.g 1 bit [0|1] (size/depth 1) directory size, 2 bits [00|01|10|11] (size/depth 2)
directory_records is how many records are being kept
'''

class Directory:
    
    def  __init__(self,global_depth,directory_records):
        
        self.global_depth = global_depth,
        self.directory_records = directory_records

'''
Here is stored each of the 
Hash prefix defines what hashes it takes, but is listed in decimal form
    We can see if in decimal form using the hash_funtion()
value is what bucket it is connected to
'''

class DirectoryRecord:
    
    def __init__(self,bucket, hash_prefix):
        
        self.hash_prefix = hash_prefix
        self.value = bucket

Directory --- (holds) ---> DirectoryRecord(s) ----(holds) ---> Bucket

# Hyper Parameters

In [3]:
#Here we are just stating some global variables
# How many items can be stored in our buckets
bucket_capacity = 2
#Set the number of initial buckets for ID
bucket_number = 3
# Set it to have a depth of 2
global_depth = 2

In [4]:
''' Initialization of buckets
    -local depth is how many values in an index we can store (keeping in mind 0 indexing on a list)
    -empty_spaces is how many slots are in our bucket (called as bucket capacity here, i guess. Not great naming)
    -set index to be a list that is empty, important since stored values must be a list
    -ID is a an identifier
'''
bucket1 = Bucket(local_depth = 1, empty_spaces = bucket_capacity, index = [], id = 1)
bucket2 = Bucket(local_depth = 1, empty_spaces = bucket_capacity, index = [], id = 2)
'''
Here we create a list of the directory_records since they need to be in a list for our directory to work.

They have a set hash prefix in decimal as well as an assigned bucket
'''
directory_records = list()  
directory_records.append(DirectoryRecord(hash_prefix = 0, bucket = bucket1))
directory_records.append(DirectoryRecord(hash_prefix = 1, bucket = bucket2))
'''
Finally we add the directory record lists to a directory object as well as setting our globabl depth to be 1
since this is a binary depth (2 directory_records)
'''
directory = Directory(global_depth = 1, directory_records = directory_records)

# Insertion Algorithm
Here we are using this to insert variables, it is important to know the steps:
1. Analyse data - Data elements may exist in many forms e.g int, sting float. So we look at the binary
2. Initialise the bucket depth and the global depth of the directories
3. Convert into a binary format
4. Consider the “Global Depth” number of the least significant bits (LSBs) (bits at the end) of data
5. Map the data according to the ID of a directory
6. Check the following conditions if a bucket overflows (Exceeds size limit)
- Global depth == Bucket depth (local depth) split the bucket into two and increment the global depth and the bucket's depth. Re-hash the elements that were present in the split bucket
- Global depth > Bucket depth (local depth): Split the bucket into two and increment the bucket depth only. Re-hash the elements that were present in the split bucket
7. Repeat the process for each element.


In [5]:
def insert(index):
    
    #set the directoy and the bucket number (global since it calls from outside the function)
    global directory
    global bucket_number
    '''
    STEP 3
    '''
    #Take the first variable in the index and hash it
    t_id = index[0]
    hash_key = hash_funtion(int(t_id))
    
    '''
    STEP 4
    '''
    #This finds what hash directory matches the inserted value
    hash_prefix = int(hash_key[-directory.global_depth[0]:], 2)

    '''
    STEP 5
    '''
    #This matches the directory to the bucket based on the hash value
    bucket = directory.directory_records[hash_prefix].value
    # Add the value to the bucket
    bucket.index.append(index)
    # Since we put an item inside the bucket we lose a space within it
    bucket.empty_spaces = int(bucket.empty_spaces)-1

    '''
    STEP 6
    '''
    # This only occurs when there is an overflow
    if(bucket.empty_spaces < 0):
        
        # Create a way to store the bucket index
        tempopary_memory = bucket.index   
        # Reset the empty spaces 
        bucket.empty_spaces = bucket_capacity
        # Clear its sotred values
        bucket.index = []

        '''
        STEP 6A 
        occurs if the depth is larger than bucket depth, meaning we only need to split the bucket
        '''
        if (directory.global_depth[0] > bucket.local_depth):

            # NUMBER OF LINKED BUCKETS
            number_of_links = 2*(directory.global_depth[0] - bucket.local_depth)
            # Increase the depth of the bucket by 1
            bucket.local_depth = bucket.local_depth + 1
            number_of_modify_links = number_of_links/2 

            #Create new bucket
            new_bucket = Bucket(local_depth = bucket.local_depth, index=[], empty_spaces = bucket_capacity, id = bucket_number)

            #Go through each record 
            for directory_record in directory.directory_records:
                #Check for the bucket
                if(directory_record.value == bucket):
                    #Each number of links to buckets is then lowered by 1 untill a the new bucket is found
                    if(number_of_modify_links != 0):
                        number_of_modify_links = number_of_modify_links - 1
                    #When it is found link the new bucket and increase the amount of buckets
                    else:
                        directory_record.value = new_bucket
                        bucket_number = bucket_number + 1

            # Insert the temprorary bucket back into the list
            for i in range(len(tempopary_memory)):
                insert(tempopary_memory[i])
                
            '''
            STEP 6B 
            Occurs when the bucket is the same size as the depth since depth needs to be increased with bucket split
            '''
        elif (directory.global_depth[0] == bucket.local_depth):
            # Create a new length for the directory that is double (since we work in binary)
            new_directory_len = 2 * len(directory.directory_records)
            # Set a new cleared records list
            new_directory_records = []

            # Iterate through the length of new directory
            for directory_record_number in range(new_directory_len):
                
                new_directory_records.append(DirectoryRecord(hash_prefix=directory_record_number,bucket=Bucket(local_depth=1,index=[],empty_spaces=bucket_capacity,id=bucket_number)))
                # Increase the number of buckets for ID
                bucket_number = bucket_number + 1
            
            # Create the new directory
            new_directory = Directory(global_depth=directory.global_depth[0]+1,directory_records=new_directory_records)

            # REHASING

            #iterate through records
            for directory_record in directory.directory_records:
                #Adjust the hashes for previous entries 
                haskey1 = '0'+hash_funtion(directory_record.hash_prefix)
                haskey2 = '1'+hash_funtion(directory_record.hash_prefix)
                #Crete new hash indexes for them
                new_index1 = int(haskey1[-directory.global_depth[0]:],2)
                new_index2 = int(haskey2[-directory.global_depth[0]:],2)
                #Add the doubled directroeis to the new dorectory
                new_directory.directory_records[new_index1].value = directory_record.value
                new_directory.directory_records[new_index2].value = directory_record.value
            #Set the directy as the new one we had created
            directory= new_directory

            # We now re-add each value form temporary memory in again with the adjusted hash index. Recursivly
            for i in range(len(tempopary_memory)):

                insert(tempopary_memory[i])
    

# A - Simple use case

In [6]:
t_id = 0
t_amount = 100
u_name = 'David'

insert([t_id, t_amount, u_name])
t_id = 1
t_amount = 101
u_name = 'David2'
insert([t_id, t_amount, u_name])

t_id = 2
t_amount = 102
u_name = 'David2'
insert([t_id, t_amount, u_name])

t_id = 4
t_amount = 102
u_name = 'David2'
insert([t_id, t_amount, u_name])

for i in range(len(directory.directory_records)):
    print("Hash Prefix: ",hash_funtion(directory.directory_records[i].hash_prefix))
    print("Bucket ID: ",directory.directory_records[i].value.id)
    print("Local Depth: ",directory.directory_records[i].value.local_depth)
    print("Stored Values: ",directory.directory_records[i].value.index)

Hash Prefix:  0000000000000000
Bucket ID:  1
Local Depth:  1
Stored Values:  [[0, 100, 'David'], [4, 102, 'David2']]
Hash Prefix:  0000000000000001
Bucket ID:  2
Local Depth:  1
Stored Values:  [[1, 101, 'David2']]
Hash Prefix:  0000000000000010
Bucket ID:  5
Local Depth:  1
Stored Values:  [[2, 102, 'David2']]
Hash Prefix:  0000000000000011
Bucket ID:  6
Local Depth:  1
Stored Values:  []


In [7]:
t_id = 8
t_amount = 102
u_name = 'David2'
insert([t_id, t_amount, u_name])

for i in range(len(directory.directory_records)):
    print("Hash Prefix: ",hash_funtion(directory.directory_records[i].hash_prefix))
    print("Bucket ID: ",directory.directory_records[i].value.id)
    print("Local Depth: ",directory.directory_records[i].value.local_depth)
    print("Stored Values: ",directory.directory_records[i].value.index)

Hash Prefix:  0000000000000000
Bucket ID:  1
Local Depth:  2
Stored Values:  [[0, 100, 'David'], [8, 102, 'David2']]
Hash Prefix:  0000000000000001
Bucket ID:  2
Local Depth:  1
Stored Values:  [[1, 101, 'David2']]
Hash Prefix:  0000000000000010
Bucket ID:  5
Local Depth:  1
Stored Values:  [[2, 102, 'David2']]
Hash Prefix:  0000000000000011
Bucket ID:  6
Local Depth:  1
Stored Values:  []
Hash Prefix:  0000000000000100
Bucket ID:  11
Local Depth:  1
Stored Values:  [[4, 102, 'David2']]
Hash Prefix:  0000000000000101
Bucket ID:  12
Local Depth:  1
Stored Values:  []
Hash Prefix:  0000000000000110
Bucket ID:  13
Local Depth:  1
Stored Values:  []
Hash Prefix:  0000000000000111
Bucket ID:  14
Local Depth:  1
Stored Values:  []


# A - Import Random And CSV

In [8]:
bucket1 = Bucket(local_depth = 1, empty_spaces = bucket_capacity, index = [], id = 1)
bucket2 = Bucket(local_depth = 1, empty_spaces = bucket_capacity, index = [], id = 2)
directory_records = list()  
directory_records.append(DirectoryRecord(hash_prefix = 0, bucket = bucket1))
directory_records.append(DirectoryRecord(hash_prefix = 1, bucket = bucket2))
directory = Directory(global_depth = 1, directory_records = directory_records)

In [9]:
#13 random numbers like the task before (part 1)
sampleNumbers = random.sample(range(1, 30), 13)
print(sampleNumbers)

[17, 24, 12, 21, 26, 6, 10, 3, 7, 11, 9, 4, 27]


In [10]:
np.savetxt("randomNumber.csv", sampleNumbers, delimiter=",", fmt='%s', header='Numbers')

In [11]:
file = open("randomNumber.csv", "r")
importData = list(csv.reader(file, delimiter=","))
file.close()
importData.pop(0)
print(importData)

[['17'], ['24'], ['12'], ['21'], ['26'], ['6'], ['10'], ['3'], ['7'], ['11'], ['9'], ['4'], ['27']]


In [12]:
for i in range(len(importData)):
    insert(importData[i])

In [13]:
print("Global Depth: ", directory.global_depth)
print("Count of directory records: ", len(directory.directory_records))
for i in range(len(directory.directory_records)):
    print("Hash Prefix: ",hash_funtion(directory.directory_records[i].hash_prefix))
    print("Bucket ID: ",directory.directory_records[i].value.id)
    print("Local Depth: ",directory.directory_records[i].value.local_depth)
    print("Stored Values: ",directory.directory_records[i].value.index)

Global Depth:  (4,)
Count of directory records:  16
Hash Prefix:  0000000000000000
Bucket ID:  1
Local Depth:  1
Stored Values:  [['24'], ['12']]
Hash Prefix:  0000000000000001
Bucket ID:  2
Local Depth:  2
Stored Values:  [['17'], ['9']]
Hash Prefix:  0000000000000010
Bucket ID:  17
Local Depth:  2
Stored Values:  [['26'], ['10']]
Hash Prefix:  0000000000000011
Bucket ID:  18
Local Depth:  3
Stored Values:  [['3']]
Hash Prefix:  0000000000000100
Bucket ID:  23
Local Depth:  1
Stored Values:  [['4']]
Hash Prefix:  0000000000000101
Bucket ID:  24
Local Depth:  1
Stored Values:  [['21']]
Hash Prefix:  0000000000000110
Bucket ID:  25
Local Depth:  1
Stored Values:  [['6']]
Hash Prefix:  0000000000000111
Bucket ID:  26
Local Depth:  1
Stored Values:  [['7']]
Hash Prefix:  0000000000001000
Bucket ID:  35
Local Depth:  1
Stored Values:  []
Hash Prefix:  0000000000001001
Bucket ID:  36
Local Depth:  1
Stored Values:  []
Hash Prefix:  0000000000001010
Bucket ID:  37
Local Depth:  1
Stored Valu

# B - Modification - Simple refactor

In [14]:
def printDirectory(index):
    print("Global Depth: ", index.global_depth)
    print("Count of directory records: ", len(index.directory_records))
    for i in range(len(index.directory_records)):
        print("Hash Prefix: ",hash_funtion(index.directory_records[i].hash_prefix))
        print("Bucket ID: ",index.directory_records[i].value.id)
        print("Local Depth: ",index.directory_records[i].value.local_depth)
        print("Stored Values: ",index.directory_records[i].value.index)

In [15]:
printDirectory(directory)

Global Depth:  (4,)
Count of directory records:  16
Hash Prefix:  0000000000000000
Bucket ID:  1
Local Depth:  1
Stored Values:  [['24'], ['12']]
Hash Prefix:  0000000000000001
Bucket ID:  2
Local Depth:  2
Stored Values:  [['17'], ['9']]
Hash Prefix:  0000000000000010
Bucket ID:  17
Local Depth:  2
Stored Values:  [['26'], ['10']]
Hash Prefix:  0000000000000011
Bucket ID:  18
Local Depth:  3
Stored Values:  [['3']]
Hash Prefix:  0000000000000100
Bucket ID:  23
Local Depth:  1
Stored Values:  [['4']]
Hash Prefix:  0000000000000101
Bucket ID:  24
Local Depth:  1
Stored Values:  [['21']]
Hash Prefix:  0000000000000110
Bucket ID:  25
Local Depth:  1
Stored Values:  [['6']]
Hash Prefix:  0000000000000111
Bucket ID:  26
Local Depth:  1
Stored Values:  [['7']]
Hash Prefix:  0000000000001000
Bucket ID:  35
Local Depth:  1
Stored Values:  []
Hash Prefix:  0000000000001001
Bucket ID:  36
Local Depth:  1
Stored Values:  []
Hash Prefix:  0000000000001010
Bucket ID:  37
Local Depth:  1
Stored Valu

# Part 3 External Sort

In [16]:

# Create the data we are working with:
# How many samples of data
sampleCount = 100
# Out size of N, 4 as per example
pageSize = 4
# Calculate how many pages of data that we will need
pageCount = math.ceil(sampleCount / pageSize)
# This is out simulated storage of disk, as a list
diskList = []
# This is our sorting heap / buffer
sorting = []
# 
diskOut = []

# Generate our samples
samples = random.sample(range(1, 200), sampleCount)
print("Our random sample data: \n{}".format(samples))

#run through and split the data into pages
for i in range(pageCount):
    if not len(samples):
        break
    samplePage = []
    for j in range(pageSize):
        if not len(samples):
            break
        samplePage.append(samples.pop())
    diskList.append(samplePage)

print("Disk data being split into pages = \n{}".format(diskList))

# Sort all values within each page
for i in range(0, len(diskList)):
    initalSort = np.sort(diskList[i])
    diskList[i] = initalSort.tolist()

print("Sort each page: \n{}".format(diskList))

# Order each page based on the first value
diskList = sorted(diskList, key=lambda x : x[0])

print("Now the pages are ordered based on first val: \n{}".format(diskList))

# Merge by adding each page to the heap
for i in range(len(diskList)):
    for j in range(len(diskList[i])):
        heap = heapq.heappush(sorting, (int(diskList[i][j]), i))

diskList=[]
# Loaded into the heap we can take the minimum value
for x in range(sampleCount):
        # Get the minimum val and append it
        root = heapq.heappop(sorting)
        diskList.append(int(root[0]))

print("Final sorted data on 'disk': \n{}".format(diskList))

Our random sample data: 
[139, 199, 4, 151, 182, 45, 85, 114, 171, 132, 60, 63, 74, 111, 16, 69, 61, 148, 180, 157, 190, 191, 145, 143, 130, 5, 138, 7, 91, 127, 73, 82, 31, 100, 152, 55, 137, 165, 35, 36, 84, 38, 97, 77, 150, 172, 173, 175, 56, 195, 25, 93, 188, 81, 178, 12, 95, 146, 34, 62, 94, 142, 42, 181, 18, 163, 28, 44, 101, 153, 185, 194, 197, 134, 78, 1, 15, 177, 170, 21, 167, 30, 65, 122, 176, 119, 70, 49, 109, 13, 29, 103, 68, 8, 162, 106, 6, 193, 133, 115]
Disk data being split into pages = 
[[115, 133, 193, 6], [106, 162, 8, 68], [103, 29, 13, 109], [49, 70, 119, 176], [122, 65, 30, 167], [21, 170, 177, 15], [1, 78, 134, 197], [194, 185, 153, 101], [44, 28, 163, 18], [181, 42, 142, 94], [62, 34, 146, 95], [12, 178, 81, 188], [93, 25, 195, 56], [175, 173, 172, 150], [77, 97, 38, 84], [36, 35, 165, 137], [55, 152, 100, 31], [82, 73, 127, 91], [7, 138, 5, 130], [143, 145, 191, 190], [157, 180, 148, 61], [69, 16, 111, 74], [63, 60, 132, 171], [114, 85, 45, 182], [151, 4, 199, 1